In [ ]:
import numpy as np
import pandas as pd
import urllib.request
from ultralytics import YOLO
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences


/Users/heonyounglee/Desktop/ocr/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [ ]:
vet_list = pd.read_csv('./fulldata_02_03_01_P_동물병원.csv', encoding='euc-kr').loc[:, ['영업상태명', '사업장명', '도로명전체주소']]
vet_list = vet_list.loc[vet_list['영업상태명'] == '영업/정상']

vet_list


In [ ]:
vet_list[vet_list['도로명전체주소'].isnull()]

In [6]:
vet_list.columns

Index(['영업상태명', '사업장명', '도로명전체주소'], dtype='object')

In [ ]:
# row 생략 없이 출력
# pd.set_option('display.max_rows', None)
vet_name_label = vet_list.loc[:, ['사업장명']]
vet_name_label.rename(columns={'사업장명':'text'}, inplace=True)
vet_name_label['category'] = 'vet_name'

# vet_list_label.columns

# vet_list_label = vet_list_label.iloc[:, 1:]
vet_name_label

In [ ]:
vet_addr_label = vet_list.loc[:, ['도로명전체주소']]
vet_addr_label = vet_addr_label[vet_addr_label['도로명전체주소'].notnull()]
vet_addr_label.rename(columns={'도로명전체주소':'text'}, inplace=True)
vet_addr_label['category'] = 'vet_address'

vet_addr_label

In [ ]:
treat_list = pd.read_csv('./mySaeKku_hosp_fare_list_re - mySaeKku_hosp_fare_list_re.csv', encoding='utf-8')

treat_list

In [ ]:
treat_list_label = treat_list.loc[:, ['소분류']]
treat_list_label.drop_duplicates()
treat_list_label.rename(columns={'소분류':'text'}, inplace=True)
treat_list_label['category'] = 'treat_name'

treat_list_label

In [64]:
vet_list_label = pd.concat([vet_name_label, vet_addr_label, treat_list_label], ignore_index=True)

vet_list_label.columns

Index(['text', 'category'], dtype='object')

In [ ]:
vet_list_label.to_csv('./vet_list_label.csv', index=False)

In [15]:
vet_list_label = pd.read_csv('./vet_list_label.csv', encoding='utf-8')

In [ ]:
import easyocr
import pandas as pd
from gensim.models import FastText
from sklearn.preprocessing import LabelEncoder
import numpy as np

# 1. 이미지에서 텍스트 추출 (EasyOCR 사용)
def extract_text_from_image(image_path):
    reader = easyocr.Reader(['en', 'ko'])  # 언어 설정 (여기서는 영어)
    results = reader.readtext(image_path)
    extracted_text = " ".join([result[1] for result in results])
    return extracted_text

# 2. 텍스트 분류 모델 준비
def prepare_classification_model():
    # sentences = [["I", "love", "programming"], ["Python", "is", "great"], ["I", "have", "a", "dog"]]
    # labels = ["hobby", "programming", "hobby"]

    # FastText 모델 학습
    fasttext_model = FastText(sentences=vet_list_label['text'], vector_size=100, window=3, min_count=1)

    # 라벨 인코딩
    label_encoder = LabelEncoder()
    label_encoder.fit(vet_list_label['category'])
    label_mapping = {label: idx for idx, label in enumerate(label_encoder.classes_)}
    
    return fasttext_model, label_encoder, label_mapping

def classify_text(text, model, encoder, label_mapping):
    tokens = text.split()
    vectors = [model.wv[token] for token in tokens if token in model.wv]

    if not vectors:
        return "Unknown"  # 텍스트가 모델의 단어 사전에 없을 때

    vector_mean = np.mean(vectors, axis=0)

    similarities = []
    for label, idx in label_mapping.items():
        # 라벨 벡터를 계산
        label_tokens = label.split()
        label_vectors = [model.wv[token] for token in label_tokens if token in model.wv]
        
        if not label_vectors:
            similarities.append(-1)  # 라벨에 벡터가 없는 경우 유사도 낮게 설정
            continue

        label_vector_mean = np.mean(label_vectors, axis=0)
        similarity = np.dot(vector_mean, label_vector_mean) / (np.linalg.norm(vector_mean) * np.linalg.norm(label_vector_mean))
        similarities.append(similarity)

    predicted_label_index = np.argmax(similarities)
    predicted_label = [label for label, idx in label_mapping.items()][predicted_label_index]
    return predicted_label

# 3. 결과를 표로 저장
def create_dataframe(text, category):
    data = {'Extracted Text': [text], 'Category': [category]}
    df = pd.DataFrame(data)
    return df

def save_to_csv(df, file_path):
    df.to_csv(file_path, index=False)
    print(f"Results saved to {file_path}")

# 전체 실행
def main(image_path, csv_file_path):
    extracted_text = extract_text_from_image(image_path)
    print("Extracted Text:", extracted_text)
    
    fasttext_model, label_encoder, label_mapping = prepare_classification_model()
    predicted_category = classify_text(extracted_text, fasttext_model, label_encoder, label_mapping)
    print("Predicted Category:", predicted_category)
    
    df = create_dataframe(extracted_text, predicted_category)
    save_to_csv(df, csv_file_path)

# 이미지 파일 경로 및 CSV 파일 경로
image_path = '../img/2.jpg'
csv_file_path = 'classified_text.csv'

# 실행
main(image_path, csv_file_path)


In [ ]:
import easyocr
import pandas as pd
from gensim.models import FastText
from sklearn.preprocessing import LabelEncoder
import numpy as np

# 1. 이미지에서 텍스트 추출 (EasyOCR 사용)
def extract_text_from_image(image_path):
    reader = easyocr.Reader(['en', 'ko'])  # 언어 설정 (여기서는 영어)
    results = reader.readtext(image_path)
    extracted_text = " ".join([result[1] for result in results])
    return extracted_text

# 2. 텍스트 분류 모델 준비
def prepare_classification_model():
    # sentences = [["I", "love", "programming"], ["Python", "is", "great"], ["I", "have", "a", "dog"]]
    # labels = ["hobby", "programming", "hobby"]

    # FastText 모델 학습
    fasttext_model = FastText(sentences=vet_list_label['text'], vector_size=100, window=3, min_count=1)

    # 라벨 인코딩
    label_encoder = LabelEncoder()
    label_encoder.fit(vet_list_label['category'])
    label_mapping = {label: idx for idx, label in enumerate(label_encoder.classes_)}
    
    return fasttext_model, label_encoder, label_mapping

def classify_text(text, model, encoder, label_mapping):
    tokens = text.split()
    vectors = [model.wv[token] for token in tokens if token in model.wv]

    if not vectors:
        return "Unknown"  # 텍스트가 모델의 단어 사전에 없을 때

    vector_mean = np.mean(vectors, axis=0)

    similarities = []
    for label in encoder.classes_:
        label_tokens = label  # 라벨을 문자열로 처리
        label_vectors = [model.wv[token] for token in label_tokens if token in model.wv]

        if not label_vectors:
            similarities.append(-1)  # 라벨에 벡터가 없는 경우 유사도 낮게 설정
            continue
        
        label_vector_mean = np.mean(label_vectors, axis=0)
        similarity = np.dot(vector_mean, label_vector_mean) / (np.linalg.norm(vector_mean) * np.linalg.norm(label_vector_mean))
        similarities.append(similarity)

    predicted_label_index = np.argmax(similarities)
    return encoder.classes_[predicted_label_index]  # 숫자가 아닌 텍스트로 반환

# 3. 결과를 표로 저장
def create_dataframe(text, category):
    data = {'Extracted Text': [text], 'Category': [category]}
    df = pd.DataFrame(data)
    return df

def save_to_csv(df, file_path):
    df.to_csv(file_path, index=False)
    print(f"Results saved to {file_path}")

# 전체 실행
def main(image_path, csv_file_path):
    extracted_text = extract_text_from_image(image_path)
    print("Extracted Text:", extracted_text)
    
    fasttext_model, label_encoder, label_mapping = prepare_classification_model()
    predicted_category = classify_text(extracted_text, fasttext_model, label_encoder, label_mapping)
    print("Predicted Category:", predicted_category)
    
    df = create_dataframe(extracted_text, predicted_category)
    save_to_csv(df, csv_file_path)

# 이미지 파일 경로 및 CSV 파일 경로
image_path = '../img/2.jpg'
csv_file_path = 'classified_text.csv'

# 실행
main(image_path, csv_file_path)


In [4]:
# 라벨 인코딩
label_encoder = LabelEncoder()
vet_list_label['category'] = label_encoder.fit_transform(vet_list_label['category'])

# 텍스트 토큰화
tokenizer = Tokenizer()
tokenizer.fit_on_texts(vet_list_label['text'])
sequences = tokenizer.texts_to_sequences(vet_list_label['text'])

# 시퀀스 패딩
max_sequence_length = max(len(x) for x in sequences)
data_padded = pad_sequences(sequences, maxlen=max_sequence_length)

In [5]:
from gensim.models import FastText

# FastText 모델 학습
fasttext_model = FastText(sentences=[text.split() for text in vet_list_label['text']], vector_size=100, window=3, min_count=1)

# 단어 임베딩 매트릭스 생성
embedding_matrix = np.zeros((len(tokenizer.word_index) + 1, 100))
for word, i in tokenizer.word_index.items():
    embedding_matrix[i] = fasttext_model.wv[word]

In [ ]:
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense

# LSTM 모델 정의
model = Sequential()
model.add(Embedding(input_dim=len(tokenizer.word_index) + 1, output_dim=100, weights=[embedding_matrix], input_length=max_sequence_length, trainable=False))
model.add(LSTM(units=128))
model.add(Dense(units=3, activation='softmax'))  # 3개의 카테고리

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# 모델 학습
X = data_padded
y = vet_list_label['category'].values
model.fit(X, y, epochs=10, batch_size=2)

In [ ]:
# 예측
predictions = model.predict(X)
predicted_categories = np.argmax(predictions, axis=1)

# 결과 출력
print("Original categories:", y)
print("Predicted categories:", predicted_categories)

In [ ]:
# 인코딩을 위한 단어 사이 공백 추가
replace_str = ['가축', '병원', '동물', '메디컬', '센터', '의료', '고양이', '애견', '안과', '치과', '피부', '노령', '심장', '외과', '수술', '암', '관절', '파충류', '전문', '출장', '진료', '종합', '펫', '수의', '클리닉', '24시', '25시', '양·한방', '한방', '부설', '응급', '의원', '임상병리', '축산', '농협', '축협', '협동조합']

for i in replace_str:
    vet_list_label['사업장명'] = vet_list_label['사업장명'].str.replace(i, f' {i} ')

vet_list_label['사업장명'] = vet_list_label['사업장명'].str.strip()
vet_list_label


In [6]:
X_data = vet_list_label['사업장명']
y_data = vet_list_label['업종']

len(X_data), len(y_data)

(5208, 5208)

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X_data, y_data, test_size=0.2, random_state=0, stratify=y_data)

tokenizer = Tokenizer(num_words = 2740 -2072 +1) # total_cnt, rare_cnt -> 2782, 2149
tokenizer.fit_on_texts(X_train)
X_train_encoded = tokenizer.texts_to_sequences(X_train)

X_train_encoded[:5]

[[1, 2], [173, 1, 2], [1, 2], [1, 6, 3], [12, 3, 1, 2]]

In [ ]:
word_to_index = tokenizer.word_index

word_to_index

In [ ]:
threshold = 2
total_cnt = len(word_to_index)
rare_cnt = 0
total_freq = 0
rare_freq = 0

for key, value in tokenizer.word_counts.items():
    total_freq = total_freq + value

    if(value < threshold):
        rare_cnt += 1
        rare_freq += 1

print('등장 빈도가 %s번 이하인 희귀 단어의 수: %s'%(threshold - 1, rare_cnt))
print("단어 집합(vocabulary)에서 희귀 단어의 비율:", (rare_cnt / total_cnt)*100)
print("전체 등장 빈도에서 희귀 단어 등장 빈도 비율:", (rare_freq / total_freq)*100)

In [10]:
vocab_size = len(word_to_index) + 1
vocab_size

2741

In [ ]:
print('상호의 최대 길이 : %d' % max(len(sample) for sample in X_train_encoded))
print('상호의 평균 길이 : %f' % (sum(map(len, X_train_encoded))/len(X_train_encoded)))
plt.hist([len(sample) for sample in X_data], bins=50)
plt.xlabel('length of samples')
plt.ylabel('number of samples')
plt.show()

In [12]:
max_len = 13
X_train_padded = pad_sequences(X_train_encoded, maxlen=max_len)
X_train_padded.shape

(4166, 13)

In [ ]:
from tensorflow.keras.layers import SimpleRNN, Embedding, Dense
from tensorflow.keras.models import Sequential

embedding_dim = 32
hidden_units = 32

model = Sequential()
model.add(Embedding(vocab_size, embedding_dim))
model.add(SimpleRNN(hidden_units))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['acc'])
history = model.fit(X_train_padded, y_train, epochs=4, batch_size=64, validation_split=0.2)

In [ ]:
X_test_encoded = tokenizer.texts_to_sequences(X_test)
X_test_padded = pad_sequences(X_test_encoded, maxlen = max_len)
print("\n 테스트 정확도: %.4f" % (model.evaluate(X_test_padded, y_test)[1]))


In [ ]:
input_encoded = tokenizer.texts_to_sequences('행복 24시 동물 병원')
input_padded = pad_sequences(input_encoded, maxlen=max_len)

model.predict(input_padded)

In [ ]:
epochs = range(1, len(history.history['acc']) + 1)
plt.plot(epochs, history.history['loss'])
plt.plot(epochs, history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

In [ ]:
# 모델저장
from keras.models import load_model

model.save('rnn_model_240702.h5')

In [13]:
model = YOLO("../runs/segment/train7/weights/best.pt")

model.train(data='./receipts segmentation_240710.v7i.yolov8/data.yaml', epochs=100, imgsz=640)

# results = model('../img/ocr_pre.jpg')

New https://pypi.org/project/ultralytics/8.2.62 available 😃 Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.2.48 🚀 Python-3.9.6 torch-2.3.1 CPU (Apple M2)
engine/trainer: task=segment, mode=train, model=../runs/segment/train7/weights/best.pt, data=./receipts segmentation_240710.v7i.yolov8/data.yaml, epochs=100, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train10, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=Fa

train: Scanning /Users/heonyounglee/Desktop/ocr/segmentation/receipts segmentation_240710.v7i.yolov8/train/labels... 9 images, 0 backgrounds, 0 corrupt: 100%|██████████| 9/9 [00:00<00:00, 259.93it/s]

train: New cache created: /Users/heonyounglee/Desktop/ocr/segmentation/receipts segmentation_240710.v7i.yolov8/train/labels.cache



val: Scanning /Users/heonyounglee/Desktop/ocr/segmentation/receipts segmentation_240710.v7i.yolov8/valid/labels... 2 images, 0 backgrounds, 0 corrupt: 100%|██████████| 2/2 [00:00<00:00, 1845.27it/s]

val: New cache created: /Users/heonyounglee/Desktop/ocr/segmentation/receipts segmentation_240710.v7i.yolov8/valid/labels.cache
Plotting labels to /Users/heonyounglee/Desktop/ocr/runs/segment/train10/labels.jpg... 


...

                   all          2          6      0.972          1      0.995      0.769      0.972          1      0.995      0.704



100 epochs completed in 0.139 hours.
Optimizer stripped from /Users/heonyounglee/Desktop/ocr/runs/segment/train10/weights/last.pt, 6.8MB
Optimizer stripped from /Users/heonyounglee/Desktop/ocr/runs/segment/train10/weights/best.pt, 6.8MB

Validating /Users/heonyounglee/Desktop/ocr/runs/segment/train10/weights/best.pt...
Ultralytics YOLOv8.2.48 🚀 Python-3.9.6 torch-2.3.1 CPU (Apple M2)
YOLOv8n-seg summary (fused): 195 layers, 3258454 parameters, 0 gradients, 12.0 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:00<00:00,  3.15it/s]


                   all          2          6      0.974          1      0.995      0.796      0.974          1      0.995      0.813
                  date          2          2      0.981          1      0.995      0.697      0.981          1      0.995      0.697
             text_area          2          4      0.967          1      0.995      0.895      0.967          1      0.995       0.93
Speed: 1.7ms preprocess, 150.7ms inference, 0.0ms loss, 0.4ms postprocess per image
Results saved to /Users/heonyounglee/Desktop/ocr/runs/segment/train10


ultralytics.utils.metrics.SegmentMetrics object with attributes:

ap_class_index: array([0, 1])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x32e3b5df0>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)', 'Precision-Recall(M)', 'F1-Confidence(M)', 'Precision-Confidence(M)', 'Recall-Confidence(M)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041, 

In [11]:
import cv2
import glob
import time
import math
from ultralytics import YOLO
import matplotlib.pyplot as plt
from easyocr import Reader
import numpy as np
from PIL import ImageFont, ImageDraw, Image

images = glob.glob('../img/*')
model = YOLO('../runs/segment/train10/weights/best.pt')

reader = Reader(lang_list=['en', 'ko'])
time_list = []

for img in images:
    results = model.predict(img)
    image= cv2.imread(img)
    res_plot = results[0].plot()
    cv2.imshow('Image', res_plot)
    # cv2.waitKey(0)

    # cv2.destroyAllWindows()
    
    start = time.time()


    for i, box in enumerate(results[0].boxes):
        x1, y1, x2, y2 = map(int, box.xyxy[0])  # Bounding Box 좌표 추출
        cropped_img = image[y1:y2, x1:x2]  # 이미지 크롭

        gray = cv2.cvtColor(cropped_img, cv2.COLOR_BGR2GRAY)

        img_result = reader.readtext(gray, batch_size=10000, detail=0, paragraph=True)

        print('검출 결과:',img_result)
        
        # 5. 크롭된 이미지 저장
        # cv2.imwrite(f"cropped_image_{i}.jpg", cropped_img)
    end = time.time()
    time_list.append(end-start)

    print('소요시간:', f'{end - start} sec')
print(f'총 {len(time_list)}개의 영수증 이미지 평균 검출 시간 : {sum(time_list)/len(time_list)} sec')



image 1/1 /Users/heonyounglee/Desktop/ocr/segmentation/../img/IMG_2011.jpg: 640x256 1 date, 2 text_areas, 110.1ms
Speed: 2.2ms preprocess, 110.1ms inference, 12.2ms postprocess per image at shape (1, 3, 640, 256)
검출 결과: ['입원비(iOkg이l하)-12시간', '22,000', '22,000', '5,500', '입원-인#전편프', '5,500', '수액-수액처치료A', '11,000', '11,000', '5,500', '정맵카테터', '5,500']
검출 결과: ['사업자 등록번호 401-59-00064 병 원 명 광주동물메디어센터 원 장 : 송정은 사업자 소재지 광주광역시 광산구 북문대로 420번길 206 전   화   번 호 : 062-523-0075']
검출 결과: ['발 행 일 2023년 1월 10일 오후 5.07:30']
소요시간: 1.4413299560546875 sec

image 1/1 /Users/heonyounglee/Desktop/ocr/segmentation/../img/IMG_2005.jpg: 640x256 2 dates, 2 text_areas, 69.0ms
Speed: 1.0ms preprocess, 69.0ms inference, 3.8ms postprocess per image at shape (1, 3, 640, 256)
검출 결과: ['진료비-평일 (1Okg미만) 8,800 처치-염증치료 C 11,000 (식모 세정 드레성) 처치-주사료 23,000 (1Okg이l상 15k9미만) 주사비추가-진용 / 소염 / 해열 33,000 제(Okg이상~2Ok@미만)', '8,800', '11,000', '23,000', '33,000']
검출 결과: ['발 행 일 : 2024년 3월 26일 오후 6:27:45']
검출 결과: ['사업자 등록번호 401-59-0006

In [2]:
from gensim.models import FastText
from keras.models import load_model
import fasttext
from khaiii import KhaiiiApi
api = KhaiiiApi()

def seperate(sentence):
    sentence = str(sentence)

    result = []

    for word in api.analyze(sentence):
        for morph in word.morphs:
            result.append(morph.lex)

    return ' '.join(result)

In [31]:
import cv2
import glob
import time
import math
from ultralytics import YOLO
import matplotlib.pyplot as plt
from easyocr import Reader
import numpy as np
from PIL import ImageFont, ImageDraw, Image

# 사전에 학습된 YOLOv8모델 호출
print('YOLOv8-segmentation 모델 호출 중...')
model = YOLO('../runs/segment/train10/weights/best.pt')

# EasyOCR 모델 호출
print('EasyOCR모델 호출중...')
reader = Reader(lang_list=['en', 'ko'])

# 사전에 학습된 카테고리 분류용 fasttext모델 호출
print('카테고리 분류용 모델 호출중...')
text_model = fasttext.load_model('./receipts_fasttext_240723')

#대상 이미지 경로
images = glob.glob('../img/*')
print(f'{len(images)}장의 이미지 불러오는 중...')
# img = '../img/IMG_2016.jpg'

# 이미지에서 글자추출에 얼마나 걸리는지 계산용 시작시간 기록
start = time.time()
print('시작시간 기록:', start)

for img in images:
    # YOLOv8모델로 텍스트영역 골라내기
    print('YOLOv8 모델로 필요한 텍스트영역 추출중...')
    results = model.predict(img)
    image= cv2.imread(img)

    # 정리용 빈 리스트
    vet_name = []
    vet_address = []
    treat_name = []
    price = []
    date = []
    time_list = []


    print('검출된 이미지에 EasyOCR모델 적용중...')
    for i, box in enumerate(results[0].boxes):
        x1, y1, x2, y2 = map(int, box.xyxy[0])  # Bounding Box 좌표 추출
        cropped_img = image[y1:y2, x1:x2]  # 이미지 크롭
        
        # 1. 그레이스케일 변환
        gray = cv2.cvtColor(cropped_img, cv2.COLOR_BGR2GRAY)

        # img_result = reader.readtext(adjusted, batch_size=10000, detail=0)
        img_result = reader.readtext(gray, batch_size=10000, detail=0, paragraph=True, y_ths = -0.001)
        # img_result = reader.readtext(gray, batch_size=10000, detail=0, paragraph=True)
        print('검출 결과:',img_result)


        for result in img_result:
            category = text_model.predict(seperate(result))[0][0]
            if 'price' in category:
                price.append(result)
        
            # elif result.find('소재지') > 0:
            elif '소재지' in result:
                vet_address.append(result.split('소재지')[1].strip())
            
            # elif result.find('명') > 0 and result.find(':') > 0:
            elif '명 :' in result or '명 ' in result:
                vet_name.append(result.split('명 ')[1].strip())
        
            # elif result.find('발') > 0 and result.find('행') > 0:
            elif ('발' in result) and ('행' in result):
                date.append(result)

            elif (len(result) > 1) and (('사업자' not in result) and (':' not in result)):
                treat_name.append(result)
        # for result in img_result:
        #     if len(result) > 1 and (('번' not in result) and ('호' not in result)):
        #         category = text_model.predict(seperate(result))[0][0]
        #         if 'price' in category:
        #             price.append(result)
        #         elif 'treat_name' in category:
        #             treat_name.append(result)
        #         elif 'vet_name' in category:
        #             vet_name.append(result)
        #         elif 'vet_address' in category:
        #             vet_address.append(result)
        #         elif 'date' in category:
        #             date.append(result)

        #         print(result, ': ', category)
        

        
        # 5. 크롭된 이미지 저장
        # cv2.imwrite(f"cropped_image_{i}.jpg", cropped_img)
    print('결과물 모두 출력 >> ')
    print('동물병원 이름 :',vet_name)
    print('동물병원 주소 :',vet_address)
    print('진료명 :',treat_name)
    print('진료비 :',price)
    print('진료일 :',date)
    print('====================================================')

end = time.time()
# time_list.append(end-start)
print('종료시간 기록 :', end)

# cv2.imshow('Image', res_plot)
# cv2.waitKey(0)


print('소요시간:', f'{end - start} sec')
# cv2.destroyAllWindows()
# print(f'총 {len(images)}개의 영수증 이미지 평균 검출 시간 : {sum(time_list)/len(time_list)} sec')
print(f'총 {len(images)}개의 영수증 이미지 평균 검출 시간 : {(end - start) / len(images)} sec')

YOLOv8-segmentation 모델 호출 중...
EasyOCR모델 호출중...
카테고리 분류용 모델 호출중...
50장의 이미지 불러오는 중...
시작시간 기록: 1722236615.725686
YOLOv8 모델로 필요한 텍스트영역 추출중...

image 1/1 /Users/heonyounglee/Desktop/ocr/segmentation/../img/IMG_2039.jpg: 640x416 1 date, 2 text_areas, 150.4ms
Speed: 17.3ms preprocess, 150.4ms inference, 35.0ms postprocess per image at shape (1, 3, 640, 416)
검출된 이미지에 EasyOCR모델 적용중...
검출 결과: ['사업자 등록번호 301-09-95188', '병', '원', '명 : 정성환동물병원', '원', '장 : 정성환', '사업자 소재지 : 충청북도 청주시 청원구 공항로 5', '전  화 번 호 : 043-216-8274']
검출 결과: ['진료비', '초진', '11,000', '전해질 분석', '검사 내과(혈액) (ABL8O), GAS', '45,000', '검사 내과(혈액)', 'CRP', '38,000', '검사', '내과', '체장영 (cPLI) 정량검', '77,000', '사', 'X-ray(방사선) / 흥복부 (~1Okg)', '70,000']
검출 결과: ['발 행 일 2023년 10월 15일 오전 11:49:59']
결과물 모두 출력 >> 
동물병원 이름 : [': 정성환동물병원']
동물병원 주소 : [': 충청북도 청주시 청원구 공항로 5']
진료명 : ['진료비', '초진', '전해질 분석', '검사 내과(혈액) (ABL8O), GAS', '검사 내과(혈액)', 'CRP', '검사', '내과', '체장영 (cPLI) 정량검', 'X-ray(방사선) / 흥복부 (~1Okg)']
진료비 : ['11,000', '45,000', '38,000', '77,000'

In [59]:
text_model.predict(seperate('진료비-평일 (1Okg미만)'))

(('__label__2.vet_address',), array([    0.52476]))

In [ ]:
import json

with open('./answer.json') as f:
    js = json.loads(f.read())

df = pd.DataFrame(js)
treats = []
prices = []

for receipt in js:
    name = receipt['동물병원 이름']
    address = receipt['동물병원 주소']
    date = receipt['발행일']
    for item in receipt['진료내역']:
        # print(item.keys())
        # print(item["진료명"])
        print(item['금액'])
        treats.append(item["진료명"])
        prices.append(item["금액"])
    # record = {
    #     '병원 이름':name,
    #     '병원 주소':address,
    #     '발행일':date,
    #     '진료내역':treats
    # }
    # records.append(record)
treats.append('카테터')
prices.append('5,500')
# df = pd.DataFrame(records)
# df['진료내역'].head()
# print(treats)
# print(prices)

In [2]:

images = glob.glob('../img/*')

images

['../img/paddle_test.jpeg',
 '../img/ocr_blue_blur.jpeg',
 '../img/ocr_dora.jpeg',
 '../img/ocr_test.jpeg',
 '../img/ocr_scanner.jpeg',
 '../img/ocr_charcter.png',
 '../img/ocr_pre.jpg',
 '../img/ocr_messy.jpeg',
 '../img/ocr_test_img.jpeg']